In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision #for datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

#device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#hyper params
num_epochs = 4
batch_size = 16
learning_rate = 0.001

# transform in general do this
"""torchvision.transforms.Normalize(
      [meanOfChannel1, meanOfChannel2, meanOfChannel3] 
    , [stdOfChannel1, stdOfChannel2, stdOfChannel3] 
)"""
transform = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

# CIFAR 10
train_dataset = torchvision.datasets.CIFAR10(root='./data',
                                          train=True,
                                         transform=transform,
                                         download=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data',
                                          train=False,
                                         transform=transform,
                                        download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size = batch_size,
                                          shuffle=True) 
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size = batch_size,
                                          shuffle=False)

classes = ('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')


Files already downloaded and verified
Files already downloaded and verified


In [18]:
class ConvNet(nn.Module):
    
    # Here we describe layer we will use
    # First we must have had defined our architecture
    # Architecture will be conv1->pool->conv2->pool->fc1->fc2->fc3
    def __init__(self):
        # DONT forget this:
        super(ConvNet,self).__init__()
        # Calculate conv output: (input - filter +2Padding)/Stride +1
        # 2,2 Pool reduces size by 2
        
        self.conv1 = nn.Conv2d(3,6,5) # 3 input channels, 6 output channels, 5x5 conv2dkernel
        self.pool = nn.MaxPool2d(2,2) # kernel size is 2, stride is 2
        self.conv2 = nn.Conv2d(6,16,5) # input channel size must be like previous output kernel size

        self.fc1 = nn.Linear(16*5*5,120) # must be input calculated from above formulas
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10) # 10 output classes
        
    def forward(self,x):
        
        x = self.pool(F.relu(self.conv1(x)))# activations dont change size
        x = self.pool(F.relu(self.conv2(x)))
        
        # for out fully connected layer we have to flatten input
        x = x.view(-1,16*5*5) # -1 so torch can define that size for us
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
        #no Softmax bc it's already included in CELoss

model = ConvNet().to(device)
criterion = nn.CrossEntropyLoss() # multi class classification
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
        

In [22]:
n_total_steps = len(train_loader) #equals the number of batches

for epoch in range(num_epochs):
  for i, (images,labels) in enumerate(train_loader): #gives us data and index
    # we have to reshape this samples of images 
    # to be 16, 784
    images = images.to(device)
    labels = labels.to(device)
    
    # forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)

    # backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if(i+1)% 2000 == 0:
      print(f'epoch {epoch+1} / {num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item():.4f}')
print('Finished training')

epoch 1 / 4, step 2000/3125, loss = 1.2502
epoch 2 / 4, step 2000/3125, loss = 1.4741
epoch 3 / 4, step 2000/3125, loss = 1.3235
epoch 4 / 4, step 2000/3125, loss = 0.8851
Finished training


In [23]:
#testing not validating
with torch.no_grad():
    
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    
    for images,labels in test_loader:
        
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)

        # _ is class label
        _, predictions = torch.max(outputs,1) #returns value and index and we are interested in index or predictions
        n_samples += labels.size(0) #nuber of samples in that batch
        n_correct += (predictions == labels).sum().item()

        for i in range (batch_size):
            
            label = labels[i]
            pred = predictions[i]
            if(label == pred):
                n_class_correct[label] +=1
            n_class_samples[label] +=1
            
    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of conv net = {acc} %')
    
    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]} = {acc} %')

Accuracy of conv net = 61.81 %
Accuracy of plane = 61.4 %
Accuracy of car = 81.2 %
Accuracy of bird = 40.9 %
Accuracy of cat = 36.4 %
Accuracy of deer = 59.6 %
Accuracy of dog = 51.8 %
Accuracy of frog = 77.0 %
Accuracy of horse = 75.2 %
Accuracy of ship = 78.2 %
Accuracy of truck = 56.4 %
